# VGG-16 on CIFAR-10
This notebook is used to experiment with VGG-16 on CIFAR-10 dataset.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../../')
sys.path.append('../../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune
import src.compression.quantization as quant

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Get model
vgg_weights = tv.models.VGG16_Weights.DEFAULT
vgg16 = tv.models.vgg16(weights=vgg_weights)
# Modify the last layer to have 10 output classes (CIFAR-10 has 10 classes)
vgg16.classifier[6] = nn.Linear(4096, 10)


In [6]:
# Load the model from a checkpoint
checkpoint = torch.load("/workspace/volume/models/vgg16_cifar10.pt")
vgg16.load_state_dict(checkpoint)

<All keys matched successfully>

In [10]:
pruned_model = torch.load("/workspace/volume/models/vgg16_cifar10_pruned.pt")

In [11]:
device = general.get_device()
vgg16.to(device)
pruned_model.to(device)
print(device)

cuda


In [3]:
# Get dataset
dataset = data.supported_datasets["CIFAR-10"]

In [4]:
# Get transforms
vgg_cifar10_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Set transforms
dataset.set_transforms(vgg_cifar10_transform)

In [14]:
before_results = eval.get_results(vgg16, dataset)
plot.print_results(**before_results)

Test: 100%|██████████| 157/157 [00:26<00:00,  5.88it/s]


Test loss: 0.2076
Test score: 93.7600
Could not calculate FLOPS
==================================== RESULTS ===================================
Loss: 0.207603
Score: 93.759952
Time per data point: 10.6316 ms
Model Size: 512.33 MB
Number of parameters: 134301514
Number of MACs: 15499459200


### Pruning

In [ ]:
pruned_model = prune.magnitude_pruning_structured(vgg16, dataset, 0.5, fineTune=True, iterative_steps=5, layers=vgg16.classifier)

In [19]:
after_results = eval.get_results(pruned_model, dataset)

Test: 100%|██████████| 157/157 [00:16<00:00,  9.66it/s]


Test loss: 0.4127
Test score: 87.6493
Could not calculate FLOPS


In [17]:
general.finetune(pruned_model, dataset, target=99, max_it=10)

Train: 100%|██████████| 6250/6250 [03:02<00:00, 34.18it/s]


Train loss: 0.1665
Train score: 94.3440


Train: 100%|██████████| 6250/6250 [03:03<00:00, 34.10it/s]


Train loss: 0.1574
Train score: 94.6300


Train: 100%|██████████| 6250/6250 [03:02<00:00, 34.21it/s]


...

Train loss: 0.1135
Train score: 96.2300


Train: 100%|██████████| 6250/6250 [03:02<00:00, 34.19it/s]


Train loss: 0.1113
Train score: 96.2480


Train: 100%|██████████| 6250/6250 [03:03<00:00, 33.97it/s]

Train loss: 0.1085
Train score: 96.4120
Maximum number of iterations reached
Finetuning finished after 10 iterations


In [28]:
plot.print_before_after_results(before_results, after_results)

============================= RESULTS BEFORE & AFTER ===========================
Loss: 0.207603 -> 0.412708 (98.80%)
Score: 93.759952 -> 87.649283 (-6.52%)
Time per data point: 10.6316 ms -> 6.4725 ms (-39.12%)
Model Size: 512.33 MB -> 242.15 MB (-52.74%)
Number of parameters: 134301514 -> 63475626 (-52.74%)
Number of MACs: 15499459200 -> 3934750048 (-74.61%)


In [20]:
old_pruned_model = torch.load("/workspace/volume/models/vgg16_cifar10_pruned.pt")

In [24]:
old_results = eval.get_results(old_pruned_model, dataset)
plot.print_results(**old_results)

Test: 100%|██████████| 157/157 [00:16<00:00,  9.56it/s]


Test loss: 0.3447
Test score: 88.9630
Could not calculate FLOPS
==================================== RESULTS ===================================
Loss: 0.344703
Score: 88.962978
Time per data point: 6.5402 ms
Model Size: 242.15 MB
Number of parameters: 63475626
Number of MACs: 3934750048


In [23]:
general.train(old_pruned_model, dataset)

Train: 100%|██████████| 6250/6250 [03:03<00:00, 34.13it/s]

Train loss: 0.1723
Train score: 94.1620


(0.17232667305782234,
 94.162,
 183099.44820404053,
 29.295911712646486,
 3.6619889640808108)

In [26]:
torch.save(pruned_model, "/workspace/volume/models/vgg16_cifar10_pruned_50.pt")

### Quantization

In [5]:
# Load a new model
vgg16 = tv.models.vgg16()
vgg16.classifier[6] = nn.Linear(4096, 10)
checkpoint = torch.load("/workspace/volume/models/vgg16_cifar10.pt")
vgg16.load_state_dict(checkpoint)

<All keys matched successfully>

In [6]:
quantized_model = quant.dynamic_quantization(vgg16)

In [12]:
example_inputs = general.get_example_inputs(dataset.train_loader)

In [17]:
quantized_model.eval()
quant.fuse_modules(quantized_model)
quantized_model(example_inputs)

AttributeError: 'Conv2d' object has no attribute 'split'

: 

In [11]:
quantized_results = eval.get_results(quantized_model, dataset)
plot.print_results(**quantized_results)

Test:   0%|          | 0/157 [00:09<?, ?it/s]


NotImplementedError: Could not run 'quantized::linear_dynamic' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear_dynamic' is only available for these backends: [CPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

CPU: registered at ../aten/src/ATen/native/quantized/cpu/qlinear_dynamic.cpp:656 [kernel]
BackendSelect: fallthrough registered at ../at